# MNIST Convolutional Neural Networks Approach

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Helpers:

In [5]:
# Init weights

def init_weights(shape):
    init_randm_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_randm_dist)

In [6]:
# Init bias

def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [7]:
# Conv2d

def conv2d(x,W):
    # x --> input tensor [batch,H,W,Channels]
    # W --> kernel [filter H, filter W, Channels IN, Channels OUT]
    
    # Zero padding --> SAME
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [8]:
# Pooling

def max_pool_2by2(x):
    # x --> [batch,h,w,c]
    
    # ksize --> size of the window for each dimension of the input tensor
    # strides --> stride of the sliding window for each dimension of the input tensor
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [9]:
# Convolutional Layer

def convolutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

In [10]:
# Normal (fully connected) layer

def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W) + b

In [11]:
# Placeholders (28 x 28)

x = tf.placeholder(tf.float32,shape=[None,784])

In [12]:
# 0 to 9 --> 10

y_true = tf.placeholder(tf.float32,shape=[None,10])

In [13]:
# Layers (1 color channel)

x_image = tf.reshape(x,[-1,28,28,1])

In [14]:
# 5 x 5 covolutional layer --> 32 feature for each 5 x 5 patch

# [neurons,neurons,channel,features/output channels]

convo_1 = convolutional_layer(x_image,shape=[5,5,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [15]:
# Convolutional layer followed by a pooling layer

convo_2 = convolutional_layer(convo_1_pooling,shape=[5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [16]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

In [17]:
# Dropout, prevent overfitting

hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [18]:
y_pred = normal_full_layer(full_one_dropout,10)

In [19]:
# Loss function

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [20]:
# Optimizer

optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [21]:
init = tf.global_variables_initializer()

In [ ]:
# Init variables and run session

steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        if i%100 == 0:
            print("On step: {}".format(i))
            print("Accuracy: ")
            
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            
            batch_x_test, batch_y_true_test = mnist.test.next_batch(200)
            
            print(sess.run(acc,feed_dict={x:batch_x_test,y_true:batch_y_true_test,hold_prob:1.0}))
            
            print('\n')

On step: 0
Accuracy: 
0.075


On step: 100
Accuracy: 
0.95


On step: 200
Accuracy: 
0.945


On step: 300
Accuracy: 
0.975


On step: 400
Accuracy: 
0.965


